In [1]:
import pandas as pd
import numpy as np
import anndata
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings
import pathlib

warnings.filterwarnings('ignore')

In [2]:
from matplotlib import rc
labelsize = 6
linewidth = 0.6
rc('lines', linewidth=linewidth)
rc('axes', labelsize=labelsize, linewidth=linewidth)
rc('xtick', labelsize=labelsize)
rc('ytick', labelsize=labelsize)
rc('xtick.major', width=linewidth)
rc('ytick.major', width=linewidth)
rc('xtick.minor', width=linewidth - 0.2)
rc('ytick.minor', width=linewidth - 0.2)

In [3]:
chrom = 'chr2'
genes = ['Lhx6']
slop = 250000
n_pc = 10
resolution = 1

In [4]:
# Parameters
chrom = "chr10"
genes = ["ENSMUSG00000035620.15", "ENSMUSG00000034612.7", "ENSMUSG00000019947.10", "ENSMUSG00000048756.11", "ENSMUSG00000034453.8", "ENSMUSG00000019843.14", "ENSMUSG00000019889.10", "ENSMUSG00000087177.7", "ENSMUSG00000020121.15", "ENSMUSG00000071324.11", "ENSMUSG00000054934.10", "ENSMUSG00000038248.8", "ENSMUSG00000003746.16", "ENSMUSG00000039835.16", "ENSMUSG00000044071.8", "ENSMUSG00000019842.6", "ENSMUSG00000019775.17", "ENSMUSG00000045555.3", "ENSMUSG00000054027.8", "ENSMUSG00000039031.16", "ENSMUSG00000020038.10", "ENSMUSG00000020102.15", "ENSMUSG00000019856.14", "ENSMUSG00000015305.6", "ENSMUSG00000019945.10", "ENSMUSG00000019846.11", "ENSMUSG00000060487.7", "ENSMUSG00000009647.13", "ENSMUSG00000064065.15", "ENSMUSG00000019888.16", "ENSMUSG00000019828.13", "ENSMUSG00000020262.15", "ENSMUSG00000020034.7", "ENSMUSG00000034591.5", "ENSMUSG00000020037.15", "ENSMUSG00000039166.16", "ENSMUSG00000112743.1", "ENSMUSG00000042846.8", "ENSMUSG00000000766.18", "ENSMUSG00000035459.18", "ENSMUSG00000019970.15", "ENSMUSG00000019944.14", "ENSMUSG00000025795.8", "ENSMUSG00000009681.10", "ENSMUSG00000040675.17", "ENSMUSG00000043259.15", "ENSMUSG00000020053.18", "ENSMUSG00000020021.4", "ENSMUSG00000037990.18", "ENSMUSG00000010461.15", "ENSMUSG00000047129.11", "ENSMUSG00000044475.14", "ENSMUSG00000112117.1", "ENSMUSG00000044633.4", "ENSMUSG00000035864.14", "ENSMUSG00000036009.16", "ENSMUSG00000015889.8", "ENSMUSG00000037151.8", "ENSMUSG00000039089.15", "ENSMUSG00000020086.6", "ENSMUSG00000112430.1", "ENSMUSG00000037747.9", "ENSMUSG00000040624.17", "ENSMUSG00000059602.14", "ENSMUSG00000056758.14", "ENSMUSG00000111627.1", "ENSMUSG00000020263.14", "ENSMUSG00000063760.9", "ENSMUSG00000020015.10", "ENSMUSG00000019978.16", "ENSMUSG00000085418.1", "ENSMUSG00000015501.10", "ENSMUSG00000086915.7", "ENSMUSG00000037855.15", "ENSMUSG00000038481.13", "ENSMUSG00000098040.1", "ENSMUSG00000019838.11", "ENSMUSG00000112555.1", "ENSMUSG00000019943.10", "ENSMUSG00000040653.6", "ENSMUSG00000071369.11", "ENSMUSG00000040009.6", "ENSMUSG00000035681.7", "ENSMUSG00000019906.15", "ENSMUSG00000020231.15", "ENSMUSG00000112497.1", "ENSMUSG00000112009.1", "ENSMUSG00000020019.4", "ENSMUSG00000056073.16", "ENSMUSG00000111731.1", "ENSMUSG00000034560.6", "ENSMUSG00000019891.15", "ENSMUSG00000020154.10", "ENSMUSG00000063428.8", "ENSMUSG00000084764.1", "ENSMUSG00000033255.8", "ENSMUSG00000112607.1", "ENSMUSG00000075225.14", "ENSMUSG00000019996.17", "ENSMUSG00000000374.8", "ENSMUSG00000019916.14", "ENSMUSG00000051354.14", "ENSMUSG00000019864.14", "ENSMUSG00000032788.15", "ENSMUSG00000074802.11", "ENSMUSG00000020151.16", "ENSMUSG00000112201.1", "ENSMUSG00000087400.8", "ENSMUSG00000004356.8", "ENSMUSG00000009070.14", "ENSMUSG00000052302.6", "ENSMUSG00000113032.1", "ENSMUSG00000019767.12", "ENSMUSG00000038122.9", "ENSMUSG00000045867.10", "ENSMUSG00000111171.1", "ENSMUSG00000038240.13", "ENSMUSG00000044770.14", "ENSMUSG00000020230.15", "ENSMUSG00000038587.9", "ENSMUSG00000038594.9", "ENSMUSG00000047638.15", "ENSMUSG00000019806.13", "ENSMUSG00000019779.15", "ENSMUSG00000071317.4", "ENSMUSG00000097742.1", "ENSMUSG00000020256.14", "ENSMUSG00000112213.1", "ENSMUSG00000020250.10", "ENSMUSG00000020100.15", "ENSMUSG00000111970.1", "ENSMUSG00000019813.16", "ENSMUSG00000020196.10", "ENSMUSG00000100000.6", "ENSMUSG00000019892.13", "ENSMUSG00000049109.15", "ENSMUSG00000020096.20", "ENSMUSG00000111674.1", "ENSMUSG00000097300.1", "ENSMUSG00000001151.10", "ENSMUSG00000025402.12", "ENSMUSG00000019873.8", "ENSMUSG00000069581.12", "ENSMUSG00000038280.12", "ENSMUSG00000020064.9", "ENSMUSG00000025404.15", "ENSMUSG00000019986.17", "ENSMUSG00000050844.5", "ENSMUSG00000020067.8", "ENSMUSG00000112137.1", "ENSMUSG00000019966.18", "ENSMUSG00000025348.9", "ENSMUSG00000089959.1", "ENSMUSG00000111042.1", "ENSMUSG00000020000.7", "ENSMUSG00000037876.17", "ENSMUSG00000019852.7", "ENSMUSG00000019849.11", "ENSMUSG00000019979.12", "ENSMUSG00000038774.9", "ENSMUSG00000089934.1", "ENSMUSG00000038160.7", "ENSMUSG00000075184.3", "ENSMUSG00000020027.18", "ENSMUSG00000069539.11", "ENSMUSG00000036478.8", "ENSMUSG00000111752.1", "ENSMUSG00000019848.14", "ENSMUSG00000059901.12", "ENSMUSG00000036019.8", "ENSMUSG00000047492.5", "ENSMUSG00000038302.14", "ENSMUSG00000019894.14", "ENSMUSG00000019880.10", "ENSMUSG00000019952.15", "ENSMUSG00000111912.1", "ENSMUSG00000020124.10", "ENSMUSG00000020140.15", "ENSMUSG00000019863.7", "ENSMUSG00000069456.4", "ENSMUSG00000020029.6", "ENSMUSG00000111994.1", "ENSMUSG00000037455.16", "ENSMUSG00000087651.3", "ENSMUSG00000019866.14", "ENSMUSG00000090200.1", "ENSMUSG00000019857.7", "ENSMUSG00000039497.8", "ENSMUSG00000020080.8", "ENSMUSG00000020130.9", "ENSMUSG00000109160.1", "ENSMUSG00000112488.1", "ENSMUSG00000060044.8", "ENSMUSG00000101517.6", "ENSMUSG00000071392.6", "ENSMUSG00000112084.1", "ENSMUSG00000020024.7", "ENSMUSG00000112354.1", "ENSMUSG00000112314.1", "ENSMUSG00000020251.13", "ENSMUSG00000003227.4", "ENSMUSG00000020183.11", "ENSMUSG00000054452.10", "ENSMUSG00000112287.1", "ENSMUSG00000020212.14", "ENSMUSG00000051236.13", "ENSMUSG00000044921.5", "ENSMUSG00000112610.1", "ENSMUSG00000020178.6", "ENSMUSG00000019935.16", "ENSMUSG00000035948.13", "ENSMUSG00000058934.8", "ENSMUSG00000020089.8", "ENSMUSG00000019831.14", "ENSMUSG00000112925.1", "ENSMUSG00000112586.1", "ENSMUSG00000112433.1", "ENSMUSG00000019951.10", "ENSMUSG00000092051.2", "ENSMUSG00000001119.7", "ENSMUSG00000112206.1", "ENSMUSG00000111587.1", "ENSMUSG00000056716.10", "ENSMUSG00000112210.1", "ENSMUSG00000020044.13", "ENSMUSG00000019936.10", "ENSMUSG00000039891.6", "ENSMUSG00000047146.17", "ENSMUSG00000019762.4", "ENSMUSG00000033444.14", "ENSMUSG00000019988.7", "ENSMUSG00000037012.18", "ENSMUSG00000112730.1", "ENSMUSG00000089656.1", "ENSMUSG00000019818.15", "ENSMUSG00000001435.15", "ENSMUSG00000112647.1", "ENSMUSG00000085347.1", "ENSMUSG00000085456.2", "ENSMUSG00000033427.14", "ENSMUSG00000033416.15", "ENSMUSG00000078453.3", "ENSMUSG00000091415.5", "ENSMUSG00000020061.18", "ENSMUSG00000020229.4", "ENSMUSG00000112290.1", "ENSMUSG00000111182.1", "ENSMUSG00000038446.8", "ENSMUSG00000034771.15", "ENSMUSG00000111131.1", "ENSMUSG00000111052.1", "ENSMUSG00000112481.1", "ENSMUSG00000019917.10", "ENSMUSG00000037846.18", "ENSMUSG00000085454.1", "ENSMUSG00000111576.1", "ENSMUSG00000020003.16", "ENSMUSG00000020105.9", "ENSMUSG00000112265.1", "ENSMUSG00000091468.3", "ENSMUSG00000046916.9", "ENSMUSG00000112732.1", "ENSMUSG00000019927.6", "ENSMUSG00000091730.1", "ENSMUSG00000020241.13", "ENSMUSG00000101801.6", "ENSMUSG00000091381.3", "ENSMUSG00000100314.6", "ENSMUSG00000110831.1", "ENSMUSG00000004360.9", "ENSMUSG00000111535.1", "ENSMUSG00000020135.13", "ENSMUSG00000019890.4", "ENSMUSG00000019790.17", "ENSMUSG00000019961.17", "ENSMUSG00000019763.11", "ENSMUSG00000039116.7", "ENSMUSG00000039879.9", "ENSMUSG00000048701.13", "ENSMUSG00000112599.1", "ENSMUSG00000019802.13", "ENSMUSG00000020169.4", "ENSMUSG00000020072.15", "ENSMUSG00000091888.3", "ENSMUSG00000112741.1", "ENSMUSG00000038214.15", "ENSMUSG00000060002.14", "ENSMUSG00000019989.8", "ENSMUSG00000064181.6", "ENSMUSG00000112513.1", "ENSMUSG00000111945.1", "ENSMUSG00000008398.15", "ENSMUSG00000019913.4", "ENSMUSG00000046841.5", "ENSMUSG00000060733.13", "ENSMUSG00000112681.1", "ENSMUSG00000112266.1", "ENSMUSG00000096553.2", "ENSMUSG00000019929.16", "ENSMUSG00000051067.8", "ENSMUSG00000040249.15", "ENSMUSG00000019984.16", "ENSMUSG00000112307.1", "ENSMUSG00000110893.1", "ENSMUSG00000112019.1", "ENSMUSG00000020097.14", "ENSMUSG00000111183.1", "ENSMUSG00000112426.1", "ENSMUSG00000112573.1", "ENSMUSG00000019854.17", "ENSMUSG00000035365.15", "ENSMUSG00000111447.1", "ENSMUSG00000110794.1", "ENSMUSG00000050994.20", "ENSMUSG00000058297.16", "ENSMUSG00000069601.14", "ENSMUSG00000015755.16", "ENSMUSG00000112046.1", "ENSMUSG00000112444.1", "ENSMUSG00000019815.7", "ENSMUSG00000046822.13", "ENSMUSG00000020091.15", "ENSMUSG00000019804.12", "ENSMUSG00000097007.3", "ENSMUSG00000015202.13", "ENSMUSG00000019997.11", "ENSMUSG00000112910.1", "ENSMUSG00000071340.2", "ENSMUSG00000111918.1", "ENSMUSG00000002578.18", "ENSMUSG00000001211.15", "ENSMUSG00000019817.18", "ENSMUSG00000037542.13", "ENSMUSG00000019826.11", "ENSMUSG00000112493.1", "ENSMUSG00000035873.8", "ENSMUSG00000112928.1", "ENSMUSG00000036923.12", "ENSMUSG00000005897.14", "ENSMUSG00000112845.1", "ENSMUSG00000020164.7", "ENSMUSG00000020007.4", "ENSMUSG00000020078.16", "ENSMUSG00000112629.1", "ENSMUSG00000112327.1", "ENSMUSG00000112249.1", "ENSMUSG00000110723.1", "ENSMUSG00000032763.11", "ENSMUSG00000112304.1", "ENSMUSG00000035206.10", "ENSMUSG00000096546.2", "ENSMUSG00000112778.1", "ENSMUSG00000020175.15", "ENSMUSG00000033105.12", "ENSMUSG00000111474.1", "ENSMUSG00000112471.1", "ENSMUSG00000035835.14", "ENSMUSG00000038039.13", "ENSMUSG00000092210.1", "ENSMUSG00000111951.1", "ENSMUSG00000110874.1", "ENSMUSG00000053980.4", "ENSMUSG00000054031.13", "ENSMUSG00000020062.7", "ENSMUSG00000086241.2", "ENSMUSG00000028630.9", "ENSMUSG00000000290.13", "ENSMUSG00000019907.10", "ENSMUSG00000074657.5", "ENSMUSG00000112548.1", "ENSMUSG00000112028.1", "ENSMUSG00000112303.1", "ENSMUSG00000049764.8", "ENSMUSG00000110353.1", "ENSMUSG00000019850.11", "ENSMUSG00000109776.1", "ENSMUSG00000044581.8", "ENSMUSG00000112666.1", "ENSMUSG00000019900.19", "ENSMUSG00000110917.1", "ENSMUSG00000087256.1"]
slop = 250000
n_pc = 10
resolution = 1


In [5]:
output_dir = pathlib.Path(chrom)
output_dir.mkdir(exist_ok=True)

## Cell Meta

In [6]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)
use_clusters = [
    i.replace(' ', '_') for i in cell_tidy_data[cell_tidy_data['CellClass'].isin(['Exc', 'Inh'])]
    ['SubType'].unique() if 'Outlier' not in i
]
len(use_clusters)

145

## ATAC peaks

In [7]:
atac_peak = pd.read_msgpack('/home/hanliu/project/mouse_rostral_brain/study/DMRCluster/SubType.ATAC_peak_merged.msg')
atac_peak = atac_peak.loc[atac_peak.index.map(lambda i: i.startswith(f'Sub{chrom}_'))].copy()

## Gene Info

In [8]:
gene_meta = pd.read_csv(
    '/home/hanliu/ref/mouse/gencode/vm22/gencode.vM22.annotation.gene.flat.tsv.gz',
    index_col='gene_id', sep='\t'
)
gene_meta = gene_meta[gene_meta['chrom'] == chrom].copy()

In [9]:
exon_bed = pd.read_csv('/home/hanliu/ref/mouse/gencode/vm22/genome_anno/exon.all.bed',
                       header=None, sep='\t')
exon_bed.columns = ['chrom', 'start', 'end', 'gene_id', 'gene_name']

## DMR Info

In [10]:
with pd.HDFStore('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRInfo.h5', 'r') as hdf:
    dmr_rate = hdf['Rate']
dmr_rate = dmr_rate.loc[dmr_rate.index.map(lambda i: i.startswith(f'Sub{chrom}_'))].copy()

dmr_corr = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/DMRGeneCorr/TotalGeneDMRCorrLoop.0.3.msg'
)
dmr_corr = dmr_corr.set_index(['DMR', 'Gene'])
dmr_corr = dmr_corr.loc[dmr_corr.index.get_level_values('DMR').isin(dmr_rate.index)].copy()

dmr_bed = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalDMR.nofilter.bed',
                      sep='\t', header=None, index_col=3)
dmr_bed.columns = ['chrom', 'start', 'end']
dmr_bed = dmr_bed[dmr_bed['chrom'] == chrom].copy()

dmr_hits = anndata.read_h5ad('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalHits.HypoDMR.h5ad')
dmr_hits = dmr_hits[dmr_rate.index, :].copy()
dmr_hits = dmr_hits[:, use_clusters].copy()

dmr_annot = anndata.read_h5ad('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRAnnotation.h5ad')
dmr_annot = dmr_annot[dmr_rate.index, :].copy()

In [11]:
dmr_hits.shape[0]

244060

In [12]:
def get_gene(gene_id):
    _gene = gene_meta.loc[gene_id]
    return _gene.name, _gene['chrom'], _gene['start'], _gene['end'], _gene['strand']

## Gene's DMR clustering

In [13]:
def calculate_gene(gene_id):
    # get gene information
    gene_id, _, gene_start, gene_end, _ = get_gene(gene_id)

    # select related DMRs
    related_dmr = dmr_bed[(dmr_bed['start'] > gene_start - slop) &
                          (dmr_bed['end'] < gene_end + slop)].copy()
    
    related_dmr_rate = dmr_rate.loc[related_dmr.index, use_clusters].copy()
    related_dmr_rate.fillna(related_dmr_rate.mean(), axis=0, inplace=True)
    
    # construct Adata
    adata = anndata.AnnData(X=related_dmr_rate.values.copy(),
                            obs=pd.DataFrame([], related_dmr_rate.index),
                            var=pd.DataFrame([], related_dmr_rate.columns))
    sc.pp.scale(adata)
    sc.pp.pca(adata)
    
    pcs = adata.obsm['X_pca'][:, :n_pc]
    sc.pp.neighbors(adata, n_neighbors=int(round(np.log2(adata.shape[0]))), n_pcs=n_pc)
    sc.tl.leiden(adata, resolution=resolution)
    return adata.obs


def get_annotation(gene_id):
    gene_id, _, gene_start, gene_end, strand = get_gene(gene_id)
    tss = gene_start if strand == '+' else gene_end
    gene_cluster = calculate_gene(gene_id)
    this_corr = dmr_corr[dmr_corr.index.get_level_values('Gene') == gene_id]['Corr']
    this_corr.index = this_corr.index.droplevel('Gene')
    gene_cluster['Corr'] = gene_cluster.index.map(this_corr).fillna(0)
    
    this_dmr_bed = dmr_bed.loc[gene_cluster.index]
    dmr_center = (this_dmr_bed['end'] + this_dmr_bed['start']) / 2
    gene_length = gene_end - gene_start
    if strand == '+':
        gene_cluster['reldist_tss'] = (dmr_center - gene_start) / gene_length
    else:
        gene_cluster['reldist_tss'] = (gene_end - dmr_center) / gene_length
    gene_cluster['in_gene_body'] = (gene_cluster['reldist_tss'] > 0) & (gene_cluster['reldist_tss'] < 1)
    
    this_annot = dmr_annot[gene_cluster.index]
    annot_df = pd.DataFrame(this_annot.X.todense(), 
                 index=this_annot.obs_names, columns=this_annot.var_names)
    
    # annotate TE cols
    dna_te = annot_df.columns[20:33]
    gene_cluster['is_dna_te'] = annot_df[dna_te].sum(axis=1) != 0
    
    line_te = annot_df.columns[33:39]
    gene_cluster['is_line_te'] = annot_df[line_te].sum(axis=1) != 0
    
    ltr_te = annot_df.columns[39:45]
    gene_cluster['is_ltr_te'] = annot_df[ltr_te].sum(axis=1) != 0
    
    sine_te = annot_df.columns[45:52]
    gene_cluster['is_sine_te'] = annot_df[sine_te].sum(axis=1) != 0
    
    # this dmr within GOI's gene feature
    gene_cluster['in_intron'] = annot_df['intron'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_exon'] = annot_df['exon'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_utr3'] = annot_df['UTR3'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_utr5'] = annot_df['UTR5'].astype(bool) & gene_cluster['in_gene_body']
    
    # previous mC study
    gene_cluster['feDMR'] = annot_df['feDMR'].astype(bool)
    gene_cluster['adultDMR'] = annot_df['adultDMR'].astype(bool)
    
    
    other_profiles = []
    # DMR hypo call in each cluster
    this_hypo_hits = dmr_hits[gene_cluster.index]
    hits_df = pd.DataFrame(this_hypo_hits.X.todense(), 
                 index=this_hypo_hits.obs_names, columns=this_hypo_hits.var_names)
    hits_df.columns = hits_df.columns.map(lambda i: f'HypoDMR.{i}')
    other_profiles.append(hits_df)
    
    # DMR rate
    related_dmr_rate = dmr_rate.loc[gene_cluster['leiden'].sort_values().index, use_clusters].copy()
    related_dmr_rate.fillna(related_dmr_rate.mean(), axis=0, inplace=True)
    related_dmr_rate.columns = related_dmr_rate.columns.map(lambda i: f'DMRRate.{i}')
    other_profiles.append(related_dmr_rate)
    
    # atac peak
    atac_peak_df = atac_peak.loc[related_dmr_rate.index, use_clusters].copy()
    other_profiles.append(atac_peak_df)
    atac_peak_df.columns = atac_peak_df.columns.map(lambda i: f'ATACPeak.{i}')
    
    dmr_annotation = pd.concat([gene_cluster] + other_profiles, axis=1, sort=True)
    return dmr_annotation

In [ ]:
for gene in genes:
    print(gene)
    check_path = output_dir / f'{gene}.DMR_cluster.msg'
    if check_path.exists():
        continue
    
    dmr_annotation = get_annotation(gene)
    dmr_annotation.to_msgpack(output_dir / f'{gene}.DMR_detail.msg', compress='zlib')
    
    cluster_annotation = dmr_annotation.groupby('leiden').mean()
    cluster_annotation.to_msgpack(output_dir / f'{gene}.DMR_cluster.msg', compress='zlib')

ENSMUSG00000035620.15
ENSMUSG00000034612.7
ENSMUSG00000019947.10
ENSMUSG00000048756.11
ENSMUSG00000034453.8
ENSMUSG00000019843.14
ENSMUSG00000019889.10
ENSMUSG00000087177.7
ENSMUSG00000020121.15
ENSMUSG00000071324.11
ENSMUSG00000054934.10
ENSMUSG00000038248.8
ENSMUSG00000003746.16
ENSMUSG00000039835.16
ENSMUSG00000044071.8
ENSMUSG00000019842.6
ENSMUSG00000019775.17
ENSMUSG00000045555.3
ENSMUSG00000054027.8
ENSMUSG00000039031.16
ENSMUSG00000020038.10
ENSMUSG00000020102.15
ENSMUSG00000019856.14
ENSMUSG00000015305.6
ENSMUSG00000019945.10
ENSMUSG00000019846.11
ENSMUSG00000060487.7
ENSMUSG00000009647.13
ENSMUSG00000064065.15
ENSMUSG00000019888.16
ENSMUSG00000019828.13
ENSMUSG00000020262.15
ENSMUSG00000020034.7
ENSMUSG00000034591.5
ENSMUSG00000020037.15
ENSMUSG00000039166.16
ENSMUSG00000112743.1
ENSMUSG00000042846.8
ENSMUSG00000000766.18
ENSMUSG00000035459.18
ENSMUSG00000019970.15
ENSMUSG00000019944.14
ENSMUSG00000025795.8
ENSMUSG00000009681.10
ENSMUSG00000040675.17
ENSMUSG00000043259.15
ENS